In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import from_json
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# PACKAGES = "org.apache.spark:spark-sql-kafka-0-10_2.11:2.2.0,org.apache.kafka:kafka-clients:0.10.1.0"
PACKAGES = "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.0"
spark = SparkSession \
    .builder \
    .appName("StructuredVotes") \
    .config("spark.jars.packages", PACKAGES)\
    .getOrCreate()



In [2]:
from ejercicios.votes import TOPIC_VOTES

In [3]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("auto.offset.reset", "latest") \
  .option("subscribe", TOPIC_VOTES) \
  .load()

In [4]:
query = df \
    .withColumn("value_json", fn.from_json(col('from_json')))
    .selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING)", "value_json") \
    .writeStream \
    .outputMode("append") \
    .format("csv") \
    .option("path", "output_dir") \
    .option("checkpointLocation", '/tmp') \
    .start()

IndentationError: unexpected indent (<ipython-input-4-85d685307bdb>, line 3)

In [5]:
schema = StructType([
    StructField("Codigo", IntegerType()),
    StructField("Partido", StringType())
])
query = df \
    .selectExpr("topic", "CAST(key AS STRING)", "CAST(value AS STRING) AS value") \
    .withColumn("value_json", fn.from_json(col('value'), schema)) \
    .select('topic', 'value_json') \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

In [6]:
query.stop()

In [ ]:
query.awaitTermination()